In [164]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote
import re
from selenium import webdriver
import pandas as pd

In [165]:
URL = "http://www.njpublicschooljobs.com/applitrack-only-pages/"
page = requests.get(URL)

data = BeautifulSoup(page.content, "html.parser")

In [166]:
counties = data.find_all(class_="zfr3Q JYVBee")

In [167]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

In [168]:
def test_new_thing(start_list):
    full_dict = {'Job Title':start_list[0],
             'Job ID': start_list[1].split(": ")[1]}

    ignore_indices = ([start_list.index(x) for x in start_list 
                           if x in ["Additional Information:  Show/Hide",
                                    "Email To A Friend",
                                    "Print Version"]])
    use_list = [start_list[i] for i in range(2,len(start_list)) if i not in ignore_indices]
    header_indices = [use_list.index(x) for x in use_list if ":" in x]

    for x in range(len(header_indices)):
        ix = header_indices[x]
        ix_next = header_indices[x+1] if x != len(header_indices)-1 else len(use_list)
        #print(ix, ix_next)
        key = use_list[ix]
        values = use_list[ix+1:ix_next] if ix_next-(ix) > 2 else use_list[ix+1]
        full_dict[key] = values
        #print(key, values)
    return full_dict

In [169]:
clean_jobs_list = []
# For each county, we want to grab the link to their respective Applitrack page.
for county in counties[0:1]:
    # Clearer variable for county name
    county_name = county.text
    
    # Grab list of districts and their lists
    list_of_districts = county.findNext('ul').find_all('li')
    print(county.text)
    
    # For each district, we pull out the district name and clean the link
    for district in list_of_districts:
        # Pull district name
        pattern = r'\s*[-–]\s*APPLITRACK$'
        district_name = re.sub(pattern, '', district.text).strip()

        # Clean link
        district_link = district.find('a').get('href')
        # TODO: Some links are https://www.google.com, and some are www.google.com, and we need to account for both.
        clean_link = unquote(district_link).split("url?q=")[1].split('&sa')[0]

        print("\t",district_name, clean_link)
        appl_page = requests.get('https://www.applitrack.com/acboe/onlineapp/default.aspx?Category=Administration')
        appl_data = BeautifulSoup(appl_page.content, "html.parser")
        driver.get(clean_link)
        list_of_jobs = driver.execute_script('return window.AppliTrackListContent').text.split("Email To A Friend\nPrint Version\n\n\n\n")
        for job in list_of_jobs:
            job_info = [x.strip() for x in job.split("\n") if x!= '']
            elem_dict = test_new_thing(job_info)
#             elem_dict = {'District': district_name,
#                          'Job Title':job_info[0],
#                          'JobID':job_info[1].split(':')[1].strip()}

#             temp_dict = {job_info[x].replace(":",""):job_info[x+1] for x in range(2,len(job_info)-1,2)}
#             elem_dict.update(temp_dict)
            clean_jobs_list += [elem_dict]
        #print('\t\t','\n\t\t'.join([x['Job Title'] for x in clean_jobs_list]))


Atlantic County
	 Atlantic City School District https://www.applitrack.com/acboe/onlineapp/default.aspx?all=1
	 Egg Harbor Township Schools https://www.applitrack.com/eggharbor/onlineapp/default.aspx?all=1
	 Galloway Township Public Schools http://www.applitrack.com/galloway/onlineapp/default.aspx?all=1
	 Greater Egg Harbor Regional High School District https://www.applitrack.com/geh/onlineapp/default.aspx?all=1
	 Hamilton Township Schools https://www.applitrack.com/hamiltonnj/onlineapp/default.aspx?all=1
	 Mullica Township Schools https://w1-4.applitrack.com/mullica/onlineapp/default.aspx?all=1


In [170]:
new_df = pd.DataFrame(clean_jobs_list)
new_df

,Job Title,Job ID,Position Type:,Date Posted:,Location:,Date Available:,Closing Date:,Attachment(s):
0,DIRECTOR OF OPERATIONS,682,Administration/Director,7/20/2022,Administration Offices,Currently Available,NaN,NaN
1,2023-2024 School Year VICE PRINCIPAL (Elementa...,743,Administration/Vice Principal,5/17/2023,Districtwide,07/01/2023,05/27/2023,NaN
2,2023-2024 School Year TEACHER COORDINATOR OF I...,745,Elementary School Teaching Pre-K-8,5/19/2023,Districtwide,09/01/2023,05/29/2023,NaN
3,2023-2024 School Year TEACHER: PRESCHOOL,746,Elementary School Teaching Pre-K-8,5/24/2023,Districtwide,09/01/2023,06/03/2023,NaN
4,2023-2024 School Year TEACHER: ART (Elementary...,722,Elementary School Teaching Pre-K-8/Art Prek-12,3/9/2023,Districtwide,09/01/2023,04/16/2023,NaN
...,...,...,...,...,...,...,...,...
81,P/T PARAPROFESSIONALS (Job ID# 594),594,Support Staff,5/19/2023,Hamilton Township School District,NaN,NaN,NaN
82,Band Teacher,357,Elementary School Teaching,5/10/2023,Mullica Township School District,09/01/2023,06/02/2023,NaN
83,Maintenance and Grounds Worker,352,Maintenance/Custodial,11/18/2022,Mullica Township School District,01/27/2022,12/16/2022,Maintenance Grounds Job Description update - f...
84,School Secretary- Long Term Substitute,354,Secretarial/Clerical,1/9/2023,Mullica Township School District,02/01/2023,01/23/2023,NaN
